In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime as dt
import MySQLdb
from jqlib.alpha101 import *

In [2]:
# 1. 连接数据库，
conn = MySQLdb.connect(
     host='db-cuhk-afi.c18nldkzqsg2.us-east-1.rds.amazonaws.com',
     user='admin',
     password='cuhk1001',
     db='rl_data',
     # autocommit=True,    # 如果插入数据，， 是否自动提交? 和conn.commit()功能一致。
)
# conn = MySQLdb.connect(
#      host='localhost',
#      user='zm',
#      password='ky111111',
#      db='data',
#      # autocommit=True,    # 如果插入数据，， 是否自动提交? 和conn.commit()功能一致。
# )
print(conn)
cur = conn.cursor()

<_mysql.connection open to 'db-cuhk-afi.c18nldkzqsg2.us-east-1.rds.amazonaws.com' at 555672b97398>


In [ ]:
conn.close()

In [76]:
try:
    sql = "select count(1) from trade_his"

    cur.execute(sql)
except Exception as e:
    print("更新多条数据失败:", e)
else:
    # 如果是插入数据， 一定要提交数据， 不然数据库中找不到要插入的数据;
    conn.commit()
info = cur.fetchall()     #获取所有的查询结果
print(info)

((1107252,),)


In [3]:
def update_data(df):
    try:
        update_info = [tuple(xi) for xi in df.values] 
        update_sqli =    "update trade_his set alpha001= (%s) ,alpha004= (%s) ,alpha005= (%s) ,alpha008= (%s) ,alpha009= (%s) ,alpha018= (%s) ,alpha019= (%s) ,alpha020= (%s) ,alpha023= (%s) ,alpha024= (%s) ,alpha029= (%s) ,alpha032= (%s) ,alpha033= (%s) ,alpha034= (%s) ,alpha037= (%s) ,alpha038= (%s) ,alpha041= (%s) ,alpha042= (%s) ,alpha046= (%s) ,alpha056= (%s) ,update_flg= (%s)where stock_code = (%s) and trade_date = (%s)"
#         print(update_sqli,update_info)
        cur.executemany(update_sqli,update_info)
    except Exception as e:
        print("更新多条数据失败:", e)
        return False
    else:
        # 如果是插入数据， 一定要提交数据， 不然数据库中找不到要插入的数据;
        conn.commit()
        return True
def transform(df1,df2):
    res = pd.merge(df1,df2,how='inner', on=['stock_code', 'stock_code'])
    return res

def seriesToDf(dic,k,v):
    dict_alpha = {'stock_code':v.index,k:v.values}
    #dic[k] = pd.DataFrame(dict_alpha)
    return pd.DataFrame(dict_alpha)
def get_df_from_db(sql):
    return pd.read_sql(sql,conn)

def insert_datares(df):
    try:
        insert_info = [tuple(xi) for xi in df.values] 
        insert_sqli = "insert into trade_his values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"   
        cur.executemany(insert_sqli,insert_info)
    except Exception as e:
        print("插入多条数据失败:", e)
        return False
    else:
        # 如果是插入数据， 一定要提交数据， 不然数据库中找不到要插入的数据;
        conn.commit()
        return True

def delete_data(df):
    try:
        del_info = [tuple(xi) for xi in df.values] 
        del_sqli = "delete from trade_his where trade_date=%s and stock_code=%s"
#         print(update_sqli,update_info)
        cur.executemany(del_sqli,del_info)
    except Exception as e:
        print("删除多条数据失败:", e)
        return False
    else:
        # 如果是插入数据， 一定要提交数据， 不然数据库中找不到要插入的数据;
        print("删除成功")
        conn.commit()
        return True

In [4]:
def getAlpha(begin_date,stockList,df):

    alpha_df = alpha(begin_date,stockList)
    alpha_df = alpha_df.reset_index()
    alpha_df.rename(columns = {"index": "stock_code"},  inplace=True)
    alpha_df = alpha_df[['stock_code','alpha_001', 'alpha_018', 'alpha_029', 'alpha_038',        
                 'alpha_004', 'alpha_019', 'alpha_032', 'alpha_041',
                 'alpha_005', 'alpha_020', 'alpha_033', 'alpha_042',
                 'alpha_008', 'alpha_023', 'alpha_034', 'alpha_046',
                 'alpha_009', 'alpha_024', 'alpha_037', 'alpha_056']]
    
    df.rename(columns = {"alpha_001": "alpha001", "alpha_018":"alpha018",
                         "alpha_029": "alpha029", "alpha_038":"alpha038",
                         "alpha_004": "alpha004", "alpha_019":"alpha019",
                         "alpha_032": "alpha032", "alpha_041":"alpha041",
                         "alpha_005": "alpha005", "alpha_020":"alpha020",
                         "alpha_033": "alpha033", "alpha_042":"alpha042",
                         "alpha_008": "alpha008", "alpha_023":"alpha023",
                         "alpha_034": "alpha034", "alpha_046":"alpha046",
                         "alpha_009": "alpha009", "alpha_024":"alpha024",
                         "alpha_037": "alpha037", "alpha_056":"alpha056"
                        },  inplace=True)
    df = transform(df,alpha_df)
#     trade_date = df['trade_date']
#     stock_code = df['stock_code']
#     df.drop(labels=['trade_date'], axis=1,inplace = True)
#     df.drop(labels=['stock_code'], axis=1,inplace = True)
#     df.insert(20, 'update_flg','Y')
#     df.insert(21, 'stock_code',stock_code)
#     df.insert(22, 'trade_date',trade_date)
    df['update_flg']='Y'
    df = df.fillna(0)
    return df
   

###### begin_date = datetime.date(2018,6,19)
stockList = ['000001.XSHE','000002.XSHE']
df = pd.DataFrame({'stock_code': ['000001.XSHE','000002.XSHE'], 'trade_date': ['a', 'b']})
df = getAlpha(begin_date,stockList,df)
print(df)

In [ ]:
#获取指定日期指数代码下股票列表
begin_date = datetime.date(2007,7,1)
end_date = datetime.date(2009,12,31)
delta = datetime.timedelta(days=1)


while begin_date <= end_date: 
    sel = "select trade_date, stock_code, open_price, close_price, high_price, low_price, volume, money, avg, pre_close, returns  from trade_his where update_flg = \"N\" and trade_date = \"" + str(begin_date) + "\""
    df = get_df_from_db(sel)
    if len(df) > 0 :
#         stockList = df['stock_code'].tolist()   
#         df = getAlpha(begin_date,stockList,df)
#         res = update_data(df)
        
        del_df = df[['trade_date','stock_code']]
        delete_data(del_df)
        stockList = df['stock_code'].tolist()   
        df = getAlpha(begin_date,stockList,df)
        res = insert_datares(df)
        if not res:
            print(begin_date,"插入失败")
            break
        if res:
            print(begin_date,"插入成功")
    else:
        print(begin_date,"非交易日:插入成功")
    begin_date += delta

2007-07-01 非交易日:插入成功
删除成功


/opt/conda/lib/python3.6/site-packages/jqlib/alpha101.py:3488: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = pd.concat(result, axis=1, keys=alphas)


2007-07-02 插入成功
删除成功


/opt/conda/lib/python3.6/site-packages/jqlib/alpha101.py:3488: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = pd.concat(result, axis=1, keys=alphas)


2007-07-03 插入成功
删除成功


...

2007-10-10 插入成功
删除成功


/opt/conda/lib/python3.6/site-packages/jqlib/alpha101.py:3488: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = pd.concat(result, axis=1, keys=alphas)


2007-10-11 插入成功
删除成功


/opt/conda/lib/python3.6/site-packages/jqlib/alpha101.py:3488: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = pd.concat(result, axis=1, keys=alphas)


2007-10-12 插入成功
2007-10-13 非交易日:插入成功
2007-10-14 非交易日:插入成功
删除成功
